# Demo for Model Debugging.

This script is intended to help with debugging a model.

It imports a model, initializes a model object with given factors, sets up pseudorandom number generators, and runs one or more replications.

## Append SimOpt Path

Since the notebook is stored in simopt/notebooks, we need to append the parent simopt directory to the system path to import the necessary modules later on.

In [ ]:
import sys
from pathlib import Path

# Take the current directory, find the parent, and add it to the system path
sys.path.append(str(Path.cwd().parent))

## Model Configuration Parameters

This section defines the core parameters for the model.

To query model names, run `python scripts/list_directories.py`

In [5]:
# Import the model from the models directory
from simopt.models.mm1queue import MM1Queue

# Set fixed factors
# Setting to {} will resort to all default values.
fixed_factors = {"lambda": 3.0, "mu": 8.0}

# Initialize model
mymodel = MM1Queue(fixed_factors=fixed_factors)

In [6]:
# Check that all factors describe a simulatable model.
# Check fixed factors individually.
for key, value in mymodel.factors.items():
    print(
        f"The factor {key} is set as {value}. "
        f"Is this simulatable? {bool(mymodel.check_simulatable_factor(key))}."
    )
# Check all factors collectively.
print(
    f"Is the specified model simulatable? {bool(mymodel.check_simulatable_factors())}."
)

The factor lambda is set as 3.0. Is this simulatable? True.
The factor mu is set as 8.0. Is this simulatable? True.
The factor warmup is set as 20. Is this simulatable? True.
The factor people is set as 50. Is this simulatable? True.
The factor epsilon is set as 0.001. Is this simulatable? True.
Is the specified model simulatable? True.


In [7]:
# Create a list of RNG objects for the simulation model to use when
# running replications.

from mrg32k3a.mrg32k3a import MRG32k3a

rng_list = [MRG32k3a(s_ss_sss_index=[0, ss, 0]) for ss in range(mymodel.n_rngs)]

In [8]:
# Run a single replication of the model.
responses, gradients = mymodel.replicate(rng_list)
print("\nFor a single replication:")
print("\nResponses:")
for key, value in responses.items():
    print(f"\t {key} is {value}.")
print("\n Gradients:")
for outerkey in gradients:
    print(f"\tFor the response {outerkey}:")
    for innerkey, value in gradients[outerkey].items():
        print(f"\t\tThe gradient w.r.t. {innerkey} is {value}.")


For a single replication:

Responses:
	 avg_sojourn_time is 0.18401110532972623.
	 avg_waiting_time is 0.06170466324915891.
	 frac_cust_wait is 0.36.

 Gradients:
	For the response avg_sojourn_time:
		The gradient w.r.t. lambda is 0.0.
		The gradient w.r.t. mu is -0.027859210338154427.
		The gradient w.r.t. epsilon is nan.
		The gradient w.r.t. warmup is nan.
		The gradient w.r.t. people is nan.
	For the response avg_waiting_time:
		The gradient w.r.t. lambda is 0.0.
		The gradient w.r.t. mu is -0.012570905078083512.
		The gradient w.r.t. epsilon is nan.
		The gradient w.r.t. warmup is nan.
		The gradient w.r.t. people is nan.
	For the response frac_cust_wait:
		The gradient w.r.t. lambda is nan.
		The gradient w.r.t. mu is nan.
		The gradient w.r.t. epsilon is nan.
		The gradient w.r.t. warmup is nan.
		The gradient w.r.t. people is nan.
